In [1]:
import json
from pathlib import Path
from tqdm import tqdm

def filter_complete_samples(input_jsonl, output_jsonl, image_root):
    """
    过滤掉有缺失图片的样本，输出新的jsonl文件
    
    Args:
        input_jsonl: 输入JSONL文件路径
        output_jsonl: 输出JSONL文件路径
        image_root: 图片根目录
    """
    image_root = Path(image_root)
    
    # 统计信息
    total_samples = 0
    valid_samples = 0
    removed_samples = 0
    total_images = 0
    missing_images = 0
    
    # 先计算总行数
    with open(input_jsonl, 'r', encoding='utf-8') as f:
        total_lines = sum(1 for _ in f)
    
    # 处理数据
    with open(input_jsonl, 'r', encoding='utf-8') as f_in, \
         open(output_jsonl, 'w', encoding='utf-8') as f_out:
        
        for line in tqdm(f_in, total=total_lines, desc="处理样本"):
            try:
                data = json.loads(line.strip())
                total_samples += 1
                
                sample_id = data.get('id', '')
                has_missing = False
                
                # 检查 visual_demo 中的图片
                visual_demo = data.get('visual_demo', [])
                for img_name in visual_demo:
                    img_path = image_root / sample_id / img_name
                    total_images += 1
                    if not img_path.exists():
                        has_missing = True
                        missing_images += 1
                
                # 检查 stage_to_estimate 中的图片
                stage_to_estimate = data.get('stage_to_estimate', [])
                for img_name in stage_to_estimate:
                    img_path = image_root / sample_id / img_name
                    total_images += 1
                    if not img_path.exists():
                        has_missing = True
                        missing_images += 1
                
                # 如果所有图片都存在，写入输出文件
                if not has_missing:
                    f_out.write(line)
                    valid_samples += 1
                else:
                    removed_samples += 1
                    
            except:
                pass
    
    # 打印统计结果
    print(f"\n{'='*80}")
    print(f"处理完成!")
    print(f"{'='*80}")
    print(f"输入样本数: {total_samples}")
    print(f"输出样本数: {valid_samples}")
    print(f"移除样本数: {removed_samples}")
    print(f"保留比例: {valid_samples/total_samples*100:.2f}%" if total_samples > 0 else "N/A")
    print(f"\n总图片数: {total_images}")
    print(f"缺失图片数: {missing_images}")
    print(f"图片完整性: {(total_images-missing_images)/total_images*100:.2f}%" if total_images > 0 else "N/A")
    print(f"\n输出文件: {output_jsonl}")
    
    if removed_samples == 0:
        print(f"\n✅ 所有样本图片完整!")
    else:
        print(f"\n⚠️  移除了 {removed_samples} 个有缺失图片的样本")


# if __name__ == '__main__':
#     # 设置路径
#     INPUT_JSONL = '/home/vcj9002/jianshu/workspace/code/ProgressLM/data/raw/visual_demo/visual_h5_franka_3rgb_raw.jsonl'      # 输入JSONL文件
#     OUTPUT_JSONL = '/home/vcj9002/jianshu/workspace/code/ProgressLM/data/raw/visual_demo/visual_h5_franka_3rgb_train.jsonl'    # 输出JSONL文件
#     IMAGE_ROOT = '/home/vcj9002/jianshu/workspace/data/robomind/data/images'  # 图片根目录
    
#     # 运行过滤
#     filter_complete_samples(INPUT_JSONL, OUTPUT_JSONL, IMAGE_ROOT)

# if __name__ == '__main__':
#     # 设置路径
#     INPUT_JSONL = '/home/vcj9002/jianshu/workspace/code/ProgressLM/data/eval/visual/visual_h5_franka_3rgb_t.jsonl'      # 输入JSONL文件
#     OUTPUT_JSONL = '/home/vcj9002/jianshu/workspace/code/ProgressLM/data/eval/visual/visual_h5_franka_3rgb_test.jsonl'    # 输出JSONL文件
#     IMAGE_ROOT = '/home/vcj9002/jianshu/workspace/data/robomind/data/images'  # 图片根目录
    
#     # 运行过滤
#     filter_complete_samples(INPUT_JSONL, OUTPUT_JSONL, IMAGE_ROOT)

if __name__ == '__main__':
    # 设置路径
    INPUT_JSONL = '/home/runsheng/personal_3/qiancx/ProgressLM/data/raw/visual_demo/visual_h5_agilex_3rgb_t.jsonl'      # 输入JSONL文件
    OUTPUT_JSONL = '/home/runsheng/personal_3/qiancx/ProgressLM/data/raw/visual_demo/visual_h5_agilex_3rgb_train.jsonl'    # 输出JSONL文件
    IMAGE_ROOT = '/home/runsheng/personal_3/qiancx/Sources/datasets/robomind/images'  # 图片根目录
    
    # 运行过滤
    filter_complete_samples(INPUT_JSONL, OUTPUT_JSONL, IMAGE_ROOT)

处理样本:   3%|▎         | 1246/45276 [00:00<00:03, 12453.54it/s]

处理样本: 100%|██████████| 45276/45276 [00:03<00:00, 11737.53it/s]


处理完成!
输入样本数: 45276
输出样本数: 0
移除样本数: 45276
保留比例: 0.00%

总图片数: 482064
缺失图片数: 45276
图片完整性: 90.61%

输出文件: /home/runsheng/personal_3/qiancx/ProgressLM/data/raw/visual_demo/visual_h5_agilex_3rgb_train.jsonl

⚠️  移除了 45276 个有缺失图片的样本


In [ ]:
import json
from pathlib import Path

def fix_jsonl_camera_names(input_file: str, output_file: str = None):
    """
    修正JSONL文件中stage_to_estimate字段的相机名称
    将camera_top替换为camera_front
    
    Args:
        input_file: 输入JSONL文件路径
        output_file: 输出JSONL文件路径（默认为输入文件名_fixed.jsonl）
    """
    input_path = Path(input_file)
    
    # 如果没有指定输出文件，自动生成
    if output_file is None:
        output_file = str(input_path.parent / f"{input_path.stem}_fixed.jsonl")
    
    # 统计信息
    total_lines = 0
    modified_lines = 0
    total_frames_modified = 0
    
    corrected_lines = []
    
    print(f"📖 读取文件: {input_file}")
    
    with open(input_file, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            total_lines += 1
            line = line.strip()
            
            if not line:
                continue
            
            try:
                data = json.loads(line)
                line_modified = False
                
                # 修正 stage_to_estimate 中的相机名称
                if 'stage_to_estimate' in data and data['stage_to_estimate']:
                    modified_frames = []
                    for frame in data['stage_to_estimate']:
                        if 'camera_top' in frame:
                            # 替换 camera_top 为 camera_front
                            new_frame = frame.replace('camera_top', 'camera_front')
                            modified_frames.append(new_frame)
                            total_frames_modified += 1
                            line_modified = True
                        else:
                            modified_frames.append(frame)
                    
                    data['stage_to_estimate'] = modified_frames
                
                if line_modified:
                    modified_lines += 1
                
                # 保存修正后的数据
                corrected_lines.append(json.dumps(data, ensure_ascii=False))
                
            except json.JSONDecodeError as e:
                print(f"⚠️ 第 {line_num} 行JSON解析错误: {e}")
                corrected_lines.append(line)  # 保留原始行
    
    # 写入修正后的文件
    print(f"\n💾 保存到: {output_file}")
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(corrected_lines))
        if corrected_lines:
            f.write('\n')  # 末尾添加换行符
    
    # 输出统计信息
    print(f"\n✅ 修正完成！")
    print(f"   总行数: {total_lines}")
    print(f"   修改行数: {modified_lines}")
    print(f"   修改帧数: {total_frames_modified}")
    print(f"   修改比例: {modified_lines/total_lines*100:.1f}%")
    
    return output_file


def verify_fix(fixed_file: str):
    """
    验证修正结果，检查是否还有camera_top
    
    Args:
        fixed_file: 修正后的文件路径
    """
    print(f"\n🔍 验证修正结果: {fixed_file}")
    
    camera_top_count = 0
    camera_front_count = 0
    
    with open(fixed_file, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            
            data = json.loads(line)
            for frame in data.get('stage_to_estimate', []):
                if 'camera_top' in frame:
                    camera_top_count += 1
                if 'camera_front' in frame:
                    camera_front_count += 1
    
    print(f"   stage_to_estimate中的camera_top: {camera_top_count}")
    print(f"   stage_to_estimate中的camera_front: {camera_front_count}")
    
    if camera_top_count == 0:
        print(f"   ✅ 所有camera_top已成功替换为camera_front")
    else:
        print(f"   ⚠️ 仍有 {camera_top_count} 个camera_top未替换")


# ========== 使用示例 ==========

if __name__ == "__main__":
    # 修正文件
    input_file = "/home/runsheng/personal_3/qiancx/ProgressLM/data/eval/visual/visual_h5_agilex_3rgb_test.jsonl"
    
    # 执行修正
    output_file = fix_jsonl_camera_names(input_file)
    
    # 验证修正结果
    verify_fix(output_file)
    
    print(f"\n🎯 现在可以使用修正后的文件:")
    print(f"   {output_file}")

📖 读取文件: /home/runsheng/personal_3/qiancx/ProgressLM/data/raw/visual_demo/visual_h5_agilex_3rgb_t.jsonl

💾 保存到: /home/runsheng/personal_3/qiancx/ProgressLM/data/raw/visual_demo/visual_h5_agilex_3rgb_t_fixed.jsonl

✅ 修正完成！
   总行数: 45276
   修改行数: 45276
   修改帧数: 45276
   修改比例: 100.0%

🔍 验证修正结果: /home/runsheng/personal_3/qiancx/ProgressLM/data/raw/visual_demo/visual_h5_agilex_3rgb_t_fixed.jsonl
   stage_to_estimate中的camera_top: 0
   stage_to_estimate中的camera_front: 45276
   ✅ 所有camera_top已成功替换为camera_front

🎯 现在可以使用修正后的文件:
   /home/runsheng/personal_3/qiancx/ProgressLM/data/raw/visual_demo/visual_h5_agilex_3rgb_t_fixed.jsonl
